<h1>
<hr style=" border:none; height:3px;">
<center>Main Workflow - Seamless Stable Diffusion CS</center>
<hr style=" border:none; height:3px;">
</h1>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>

<h4><center>Louis LHOTTE | Clément VERON | Edouard SEGUIER</center></h4>

# I. Imports

# II. Observation and Blending

<div class="alert alert-block alert-info"> Après avoir effectué des tests, nous nous sommes rendus compte que certains ajouts étaient un peu folklorique et peu réaliste par rapport à l'image original et le but de la modification. Par exemple, ajouter un casque à une photo normale donne systématiquement (sur trois essais) un outlier ou un demi-casque aux couleurs peu adaptées. L'idée est donc d'ajouter dans la pipeline une étape de <b>blending</b> pour que le casque s'intègre bien dans la photo (objectif de réalisme et d'intégration)</div>